In [ ]:
pip install botorch

In [2]:
import pandas as pd
import torch

train_data = pd.read_excel('data_sample_multiple_components_withoutAverage_XYLOSE_9objs_iter01.xlsx')
#print(train_data)
train_x = train_data.iloc[:,[0,1]].values
#print(train_X)
train_x = torch.tensor(train_x, dtype=torch.float32)
print(train_x)

train_obj = train_data.iloc[:,2:].values
#print(train_Y)
train_obj = torch.tensor(train_obj, dtype=torch.float32)
print(train_obj)

tensor([[5.0000e-03, 5.0000e-01],
        [5.0000e-03, 5.0000e-01],
        [5.0000e-03, 5.0000e-01],
        [5.0000e-03, 1.0000e+00],
        [5.0000e-03, 1.0000e+00],
        [5.0000e-03, 1.0000e+00],
        [5.0000e-03, 2.0000e+00],
        [5.0000e-03, 2.0000e+00],
        [5.0000e-03, 2.0000e+00],
        [4.0600e-02, 5.0000e-01],
        [4.0600e-02, 5.0000e-01],
        [4.0600e-02, 5.0000e-01],
        [4.0600e-02, 1.0000e+00],
        [4.0600e-02, 1.0000e+00],
        [4.0600e-02, 1.0000e+00],
        [4.0600e-02, 2.0000e+00],
        [4.0600e-02, 2.0000e+00],
        [4.0600e-02, 2.0000e+00],
        [1.2088e-03, 3.8620e-01],
        [1.3619e-02, 2.5995e+00],
        [2.4758e-02, 2.5750e-01],
        [4.1170e-02, 1.5790e-01],
        [4.4772e-02, 2.0989e+00]])
tensor([[ 0.1151,  0.0591,  0.0179,  0.1398,  0.0145,  0.1592,  0.0000,  0.0449,
         -0.4976],
        [ 0.1204,  0.0640,  0.0071,  0.0992,  0.0158,  0.1706,  0.0000,  0.0681,
         -0.5348],
        [ 0.1208,

In [3]:
from botorch import fit_gpytorch_model
from botorch.models.gp_regression import FixedNoiseGP
from botorch.models.transforms.outcome import Standardize
from botorch.models.model_list_gp_regression import ModelListGP
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch.utils.transforms import unnormalize, normalize
from botorch.utils.multi_objective.box_decompositions.non_dominated import FastNondominatedPartitioning
from botorch.acquisition.multi_objective.monte_carlo import qExpectedHypervolumeImprovement
from botorch.sampling.samplers import SobolQMCNormalSampler
from botorch.optim.optimize import optimize_acqf

torch.manual_seed(0)

NOISE_SE = torch.tensor([1,1,1,1,1,1,1,1,1])
#print(NOISE_SE)

standard_bounds = torch.tensor([[0.001, 0.1],[0.05, 3]])
#print(standard_bounds)
ref_point=torch.tensor([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0])

# define models for objective and constraint
#train_x = normalize(train_x, standard_bounds)
models = []
for i in range(train_obj.shape[-1]):
    train_y = train_obj[..., i:i+1]
    train_yvar = torch.full_like(train_y, NOISE_SE[i] ** 2)
    models.append(FixedNoiseGP(train_x, train_y, train_yvar, outcome_transform=Standardize(m=1)))
model = ModelListGP(*models)
mll = SumMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_model(mll)

BATCH_SIZE = 10
NUM_RESTARTS = 2
RAW_SAMPLES = 4
MC_SAMPLES = 16

with torch.no_grad():
  #pred = model.posterior(normalize(train_x, standard_bounds)).mean
  pred = model.posterior(train_x).mean
partitioning = FastNondominatedPartitioning(ref_point, Y=pred)
sampler = SobolQMCNormalSampler(num_samples=MC_SAMPLES)
acq_func = qExpectedHypervolumeImprovement(model,ref_point,partitioning,sampler)

candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        #options={"batch_limit": 5, "maxiter": 200},
        sequential=True)
#candidates =  unnormalize(candidates.detach(), bounds=standard_bounds)
print(candidates)

/usr/local/lib/python3.7/dist-packages/gpytorch/lazy/lazy_tensor.py:1810: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2183.)
  Linv = torch.triangular_solve(Eye, L, upper=False).solution


tensor([[4.1770e-02, 1.5790e-01],
        [1.3619e-02, 2.5995e+00],
        [1.9970e-02, 2.2111e-01],
        [4.4772e-02, 2.0989e+00],
        [2.7305e-03, 2.0148e-01],
        [1.2088e-03, 3.8620e-01],
        [4.0980e-03, 6.6677e-01],
        [5.1903e-03, 2.7432e+00],
        [3.3898e-02, 2.4039e+00],
        [2.4758e-02, 2.5757e-01]])
